In [1]:
import gym
import numpy as np
import tensorflow as tf
tf.__version__

'1.5.0'

In [2]:
#CNN
def cnn(inputs_feed):
  """Model function for CNN."""
  mode = 'TRAIN'
  # Input Layer

  input_layer = tf.reshape(inputs_feed, [-1, 80,80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Dense Layer
  conv2_flat = tf.reshape(conv2, [-1, 80 * 80 * 16])
  dense = tf.layers.dense(inputs=conv2_flat, units=1024, activation=tf.nn.relu)
    
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  sigmoid = tf.layers.dense(inputs=dropout, units=1)
  up_probability = tf.nn.sigmoid(sigmoid) 

  return up_probability

#image pre processing

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    #print input_observation
    return input_observation, prev_processed_observations

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3
    # 0 is up and 1 is down
    # < 0.5 is up is 0
    

def weight_varible(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0 # erase background (background type 1)
    I[I == 109] = 0 # erase background (background type 2)
    I[I != 0] = 1 # everything else (paddles, ball) just set to 1
    return I.astype(np.float).ravel()

In [ ]:
# main

# hyperparameters
learning_rate = 0.0001
episode_number = 0
number  = 0
batch_size = 2
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0
sum_of_probability = 0
D = 80 * 80 # input dimensionality: 80x80 grid
#1 get image and reward
counter = 1
#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

# define a graph
graph1 = tf.Graph()
with graph1.as_default():
    env = gym.make("Pong-v0")
    observation = env.reset()
    prev_x = None # used in computing the difference frame
    print observation.shape
    processed_observation=observation[35:195]
    print processed_observation.shape

    a=remove_background(processed_observation)
    print a.shape
    ###################################################################
    #1 input
    x_ = tf.placeholder(shape=[None,6400],dtype=tf.float32)
    y_ = tf.placeholder(tf.float32, (None,1), name="fake_label")
    #reward_ =  tf.placeholder(shape=[1],dtype=tf.float32)
    reward_ = tf.placeholder(tf.float32, shape=(), name="reward_")
    #2 CNN(X)
    up_probability = cnn(x_)
    
    ##########################################
    '''
    """Model function for CNN."""
    mode = 'TRAIN'
    # Input Layer

    input_layer = tf.reshape(x_, [-1, 80,80, 1])
    # paras
    W_conv1 = weight_varible([5, 5, 1, 8])
    b_conv1 = bias_variable([8])
    
    h_conv1 = tf.nn.relu(conv2d(input_layer, W_conv1) + b_conv1)
    
    # conv layer-2
    W_conv2 = weight_varible([5, 5, 8, 16])
    b_conv2 = bias_variable([16])

    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)

    # full connection
    W_fc1 = weight_varible([80 * 80 * 16, 1024])
    b_fc1 = bias_variable([1024])

    h_conv2_flat = tf.reshape(h_conv2, [-1, 80 * 80 * 16])
    h_fc1 = tf.nn.relu(tf.matmul(h_conv2_flat, W_fc1) + b_fc1)

    # dropout
    h_fc1_drop = tf.nn.dropout(h_fc1, 0.6)

    # output layer: softmax
    W_fc2 = weight_varible([1024, 1])
    b_fc2 = bias_variable([1])

    up_probability = tf.nn.sigmoid(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    '''
    ##########################################

    #3 cross_entropy
    #cost =  -1 * tf.exp(  tf.reduce_sum(  -1*( ( ( (1-y_)*tf.log(up_probability) + (y_) *tf.log(1-up_probability) ) ) + tf.log( reward_ )  )  )  ) 
    #cost =  tf.reduce_mean(  tf.reduce_sum( (1-y_)*tf.log(up_probability) + (y_) *tf.log(1-up_probability) ) )
    #cost = tf.reduce_mean(-tf.reduce_sum( (1-y_)*tf.log(up_probability + tf.log( reward_ ) )   , reduction_indices=1))
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
    cost = - ( tf.reduce_sum( (1-y_)*tf.log(up_probability) + (y_)* tf.log(1 - up_probability) + tf.log( reward_ ) ) )
    #4 Optimazor
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

    #5 xs_batch, reward_batch
    #
    ###############################################################################

    init = tf.global_variables_initializer()


    '''
    processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
    #processed_observations = tf.cast(processed_observations, tf.float32)
    print "1="
    print processed_observations
    print processed_observations.shape
    print processed_observations.dtype
    processed_observations = [float(v) for v in processed_observations]
    '''
    
    ########################################################################################3
    # compute policy loss and regularization loss
    #copy from others
    #cross_entropy_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=up_probability, labels=taken_actions)
    #pg_loss  = tf.reduce_mean(self.cross_entropy_loss)
    #loss_with_reward = pg_loss * discounted_rewards

    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_with_reward)
    # compute gradients
    #gradients = self.optimizer.compute_gradients(self.loss)

    # compute policy gradients
    #for i, (grad, var) in enumerate(self.gradients):
    #if grad is not None:
    #    gradients[i] = (grad * self.discounted_rewards, var)
    ###########################################################################################

    #up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
    #Make the Magic happen
    #each_value_of_object_function =  up_probability_from_cnn # * reward
    '''
    ##################################################################################
    #Magic PG value * probability
    total_value_of_object_function = []
    total_value_of_object_function.append(each_value_of_object_function)
    # Minimize error using cross entropy
    #[tf.reduce_sum(tf.square(x)) for x in policy_network_variables]
    cost = tf.reduce_mean( -1 * reward_sum * np.sum(each_value_of_object_function, reduction_indices=1) for each_value_of_object_function in total_value_of_object_function)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    ###################################################################################
    '''
    #input to the CNN and get the output up_probability
    episode_processed_observations = []
    episode_rewards =  []
    episode_fake_label_action = []
    saver = tf.train.Saver()
    
    with tf.Session(graph=graph1) as sess:     

        init = tf.global_variables_initializer()
        sess.run(init)

        while True:
            env.render()


            # feed the CNN to get an action

            processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)

            processed_observations_np = np.reshape(processed_observations, [-1,6400])
            #print "1=" + str(processed_observations_np)
            #print "2=" + str(observation[0][10])
            #processed_observations = [0,float(v) for v in processed_observations]
            #print(processed_observations.shape)    
            
            # preprocess the observation, set input to network to be difference image
            #cur_x = prepro(observation)
            #print "2=" + str(observation)
            #print "prepro = " + str(cur_x)
            #for V in cur_x:
            #    if V > 0 :
            #        print V
            #x_input = cur_x - prev_x if prev_x is not None else np.zeros(D)
            #prev_x = cur_x
            
            #x_input = np.reshape(x_input, [-1,6400])
            #print "image = " + str(x_input[0][500])
            #counter = 0
            #for V in processed_observations_np[0]:
            #    counter += 1
            #    if V > 0 :
            #        print V
            #        print"counter = " + str(counter)
            #counter = 0
            
            up_probability_from_cnn = sess.run(up_probability,feed_dict={x_: processed_observations_np})
            #up_probability_from_cnn = [[0]]
            episode_processed_observations.append(processed_observations_np[0])
            
            
            
            print(up_probability_from_cnn)    
            #print("!!!!!!!!!!!!!!!!!!!!!!!")
            #print(np.shape(episode_processed_observations)) 
            #print(np.array(episode_processed_observations).shape)       


            action = choose_action(up_probability_from_cnn)
            
            #print("Action is ===========") + str(action)
            
            episode_fake_label_action.append( [action - 2] )
            
            #print(np.shape(episode_fake_label_action)) 
            
            # carry out the chosen action
            observation, reward, done, info = env.step(action)


            ############################
            #debug
            #print "reward = " + str(reward)
            
            reward_sum += reward
            #print "reward sum = " + str(reward_sum)


            number +=1
            #print "number = " + str(number)
            #print "episode_number = " + str(episode_number)
            #print "+++++++++++"

            #
            ####################################
            xs_batch = np.reshape( episode_processed_observations , [-1,6400])
            #print(xs_batch.shape)


            ##################################################################################
            #Magic PG value * probability
            total_value_of_object_function = []
            total_value_of_object_function.append(up_probability_from_cnn)

            if done: # an episode finished
                print "episode_number = " + str(episode_number)
                episode_number += 1

                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")
               # print(episode_processed_observations)
                #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
                ###################################################################################
                #for i in range(100):
                #   optimizer.run()
                #  print "cost_goal: ",cost.eval()

                xs_batch = episode_processed_observations
                
                #print("length of xs batch =")
                #print(np.shape(xs_batch))
                
                ys_batch = episode_fake_label_action
                
                #print("length of ys_batch =")
                #print(np.shape(ys_batch))
                
                #xs_batch = xs_batch[:400]
                #reward_batch = reward_batch[:400]
                #print(np.shape(xs_batch))
                #print np.array(reward_batch).shape
                
                num= len(episode_processed_observations) / 300
                
                if num%300 == 0 :
                    continue
                #print("number of batch = ") + str(num)
                reward_sum += 22
                
                for n in range(num+1):
                    xs_batch_temp = xs_batch[n*300:(n+1)*300]
                    ys_batch_temp = ys_batch[n*300:(n+1)*300]
                    if n == num and len(episode_processed_observations) != n*300 :
                        xs_batch_temp = xs_batch[n*300:]
                        ys_batch_temp = ys_batch[n*300:]
                    
                    #print("the traning data shape is =")
                    print(np.shape(xs_batch_temp))
                    print(np.shape(ys_batch_temp))
                
                    if  len(xs_batch_temp) ==0 or len(ys_batch_temp) ==0:
                        print("wrong")
                        continue
                    
                    _,cost_result = sess.run( [train_step, cost], feed_dict={x_: xs_batch_temp, y_ : ys_batch_temp, reward_: reward_sum} )
                    print("the run time for once COST = ") + str(cost_result)
                    print "reward sum = " + str(reward_sum)

                print("SAVED")
                #save_path = saver.save(sess, "/home/king/DeepLearning/DeepReinforcementLearning/model.ckpt")
                if episode_number % batch_size == 0:
                    if counter == 1:
                        saver.restore(sess, "/home/king/DeepLearning/DeepReinforcementLearning/model.ckpt")
                        counter +=1
                    save_path = saver.save(sess, "/home/king/DeepLearning/DeepReinforcementLearning/model.ckpt")
                    saver.restore(sess, "/home/king/DeepLearning/DeepReinforcementLearning/model.ckpt")
                    
                    
                reward_sum = 0
                xs_batch=[]
                ys_batch=[]
                reward_batch = []
                episode_processed_observations =[]
                episode_fake_label_action = []
                prev_processed_observations = None
                observation = env.reset() # reset env


    #_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})

[2018-02-26 12:10:41,909] Making new env: Pong-v0


(210, 160, 3)
(160, 160, 3)
(160, 160, 3)
[[0.5]]
[[0.5002374]]
[[0.5]]
[[0.49826202]]
[[0.50284755]]
[[0.49687338]]
[[0.49979946]]
[[0.5034721]]
[[0.50222856]]
[[0.49963418]]
[[0.49881798]]
[[0.50537825]]
[[0.5057373]]
[[0.50452733]]
[[0.50610375]]
[[0.5002778]]
[[0.50113195]]
[[0.5]]
[[0.5017041]]
[[0.5]]
[[0.49984312]]
[[0.5017041]]
[[0.502834]]
[[0.5019776]]
[[0.5019387]]
[[0.5003528]]
[[0.50001276]]
[[0.5026055]]
[[0.501825]]
[[0.5074193]]
[[0.50464094]]
[[0.50762373]]
[[0.5054012]]
[[0.5034552]]
[[0.5011534]]
[[0.5037556]]
[[0.5009583]]
[[0.49980327]]
[[0.5007574]]
[[0.49746883]]
[[0.49635547]]
[[0.50063854]]
[[0.49931964]]
[[0.50024796]]
[[0.5020911]]
[[0.5079369]]
[[0.50171906]]
[[0.5000901]]
[[0.50312835]]
[[0.5005752]]
[[0.49995923]]
[[0.502169]]
[[0.5016495]]
[[0.5028333]]
[[0.49950424]]
[[0.50207454]]
[[0.5037087]]
[[0.5056623]]
[[0.505528]]
[[0.50055856]]
[[0.50203496]]
[[0.4981588]]
[[0.501641]]
[[0.5030631]]
[[0.49816498]]
[[0.4997793]]
[[0.49811515]]
[[0.49886402]]
[[0.

In [14]:
1, -277.28204

4164.5967

In [9]:
# main

# hyperparameters
learning_rate = 0.01
episode_number = 0
number  = 0
batch_size = 3
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0
sum_of_probability = 0
#1 get image and reward
env = gym.make("Pong-v0")
observation = env.reset()
print observation.shape
processed_observation=observation[35:195]
print processed_observation.shape

a=remove_background(processed_observation)
print a.shape
#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

###################################################################
inputs_feed = tf.placeholder(shape=[6400],dtype=tf.float32)

init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#processed_observations = tf.cast(processed_observations, tf.float32)
print "1="
print processed_observations
print processed_observations.shape
print processed_observations.dtype
processed_observations = [float(v) for v in processed_observations]
up_probability = cnn(inputs_feed)


#up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
#Make the Magic happen
#each_value_of_object_function =  up_probability_from_cnn # * reward
'''
##################################################################################
#Magic PG value * probability
total_value_of_object_function = []
total_value_of_object_function.append(each_value_of_object_function)
# Minimize error using cross entropy
#[tf.reduce_sum(tf.square(x)) for x in policy_network_variables]
cost = tf.reduce_mean( -1 * reward_sum * np.sum(each_value_of_object_function, reduction_indices=1) for each_value_of_object_function in total_value_of_object_function)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
###################################################################################
'''
#input to the CNN and get the output up_probability

with tf.Session() as sess:     
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    while True:
        env.render()

        print "2="
        up_probability_from_cnn = sess.run(up_probability,feed_dict={inputs_feed: processed_observations})
        print(up_probability_from_cnn)    
        


        action = choose_action(up_probability_from_cnn)
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []
        
        ############################
        #debug
        print "reward = " + str(reward)
        reward_sum += reward
        print "reward sum = " + str(reward_sum)
        episode_rewards.append(reward)
        number +=1
        print "number = " + str(number)
        print "episode_number = " + str(episode_number)
        print "+++++++++++"
        #
        ####################################
        
        ##################################################################################
        #Magic PG value * probability
        total_value_of_object_function = []
        total_value_of_object_function.append(up_probability_from_cnn)

        sum_of_probability += up_probability_from_cnn

        if done: # an episode finished
            episode_number += 1


            optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
            ###################################################################################
            for i in range(100):
                optimizer.run()
                print "cost_goal: ",cost.eval()
            
            
            
            if episode_number % batch_size == 0:
                break
                
            observation = env.reset() # reset env
            reward_sum = 0
            prev_processed_observations = None

#_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})

[2018-01-30 14:37:42,505] Making new env: Pong-v0


(210, 160, 3)
(160, 160, 3)
(160, 160, 3)
1=
[0. 0. 0. ... 0. 0. 0.]
(6400,)
float64
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 1
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 2
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 3
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 4
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 5
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 6
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 7
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 8
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 9
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 10
episode_number = 0
+++++++++++
2=
[[0.5]]
reward = 0.0
reward sum = 0.0
number = 11
episode_number = 0
+++++++++++


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'conv2d/kernel:0' shape=(5, 5, 1, 8) dtype=float32_ref>", "<tf.Variable 'conv2d/bias:0' shape=(8,) dtype=float32_ref>", "<tf.Variable 'conv2d_1/kernel:0' shape=(5, 5, 8, 16) dtype=float32_ref>", "<tf.Variable 'conv2d_1/bias:0' shape=(16,) dtype=float32_ref>", "<tf.Variable 'dense/kernel:0' shape=(102400, 1024) dtype=float32_ref>", "<tf.Variable 'dense/bias:0' shape=(1024,) dtype=float32_ref>", "<tf.Variable 'dense_1/kernel:0' shape=(1024, 1) dtype=float32_ref>", "<tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32_ref>", "<tf.Variable 'conv2d_2/kernel:0' shape=(5, 5, 1, 8) dtype=float32_ref>", "<tf.Variable 'conv2d_2/bias:0' shape=(8,) dtype=float32_ref>", "<tf.Variable 'conv2d_3/kernel:0' shape=(5, 5, 8, 16) dtype=float32_ref>", "<tf.Variable 'conv2d_3/bias:0' shape=(16,) dtype=float32_ref>", "<tf.Variable 'dense_2/kernel:0' shape=(102400, 102

In [1]:
# main

# hyperparameters
learning_rate = 0.01
episode_number = 0
number  = 0
batch_size = 20
observation =0
reward = 0
done = 0
info = 0 
reward_sum = 0

#1 get image and reward
env = gym.make("Pong-v0")
observation = env.reset()
print observation.shape
processed_observation=observation[35:195]
print processed_observation.shape

a=remove_background(processed_observation)
print a.shape
#print a

# 2 process image
prev_processed_observations = None
input_dimensions = 80 * 80
#processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#print "1="
#print processed_observations
#print processed_observations.shape
#processed_observations = tf.cast(processed_observations, tf.float32)

###################################################################
inputs_feed = tf.placeholder(shape=[6400],dtype=tf.float32)

init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
#processed_observations = tf.cast(processed_observations, tf.float32)
print "1="
print processed_observations
print processed_observations.shape
print processed_observations.dtype
processed_observations = [float(v) for v in processed_observations]

up_probability = cnn(inputs_feed)
#input to the CNN and get the output up_probability
with tf.Session() as sess:     


    print "3="
    print up_probability
    #print the product
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(up_probability,feed_dict={inputs_feed: processed_observations}))    
    #print(up_probability.eval()[0][0])
    
#Make the Magic happen
each_value_of_object_function =  up_probability  * reward
# Minimize error using cross entropy
cost = tf.reduce_mean(-1*tf.reduce_sum(each_value_of_object_function, reduction_indices=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

#_, c = sess.run([optimizer, cost], feed_dict={x: batch_xs,y: batch_ys})
'''
init = tf.global_variables_initializer()

processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
processed_observations = tf.cast(processed_observations, tf.float32)
up_probability = cnn()



with tf.Session() as sess:     

    sess.run(init)
    
    while True:
        env.render()

        
       #up_probability = cnn(processed_observations)
        print "3="
        print episode_number
        #rint up_probability
        #print the product
        print(sess.run(up_probability,feed_dict=processed_observations))    
        print(up_probability.eval()[0][0])
        
        action = choose_action(up_probability.eval()[0][0])
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []

        reward_sum += reward
        episode_rewards.append(reward)

        number +=1
        print number
        print "==========="
        print episode_number
        print "+++++++++++"

        if done: # an episode finished
            episode_number += 1

            #if episode_number % batch_size == 0:

                #break
            observation = env.reset() # reset env
'''

NameError: name 'gym' is not defined

In [ ]:
    
    with tf.Session() as sess:     

       #up_probability = cnn(processed_observations)
        print "3="
        print episode_number
        print up_probability
        #print the product
        init = tf.global_variables_initializer()
        sess.run(init)
        print(sess.run(up_probability))    
        print(up_probability.eval()[0][0])

        action = choose_action(up_probability.eval()[0][0])
        # carry out the chosen action
        observation, reward, done, info = env.step(action)
        episode_rewards =  []

        reward_sum += reward
        episode_rewards.append(reward)

In [3]:
#CNN
def cnn(inputs_feed):
  """Model function for CNN."""
  mode = 'TRAIN'
  # Input Layer

  input_layer = tf.reshape(inputs_feed, [-1, 80,80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=8,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=16,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Dense Layer
  conv2_flat = tf.reshape(conv2, [-1, 80 * 80 * 16])
  dense = tf.layers.dense(inputs=conv2_flat, units=1024, activation=tf.nn.relu)
    
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.6, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  sigmoid = tf.layers.dense(inputs=dropout, units=1)
  up_probability = tf.nn.sigmoid(sigmoid)

  return up_probability

In [7]:
a=[]
a.append(1)
a.append(2)
a.append(3)
print np.vstack(a)
print np.vstack(a)
print np.vstack(a)
print a
print np.vstack(a) * np.vstack(a)

[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[1, 2, 3]
[[1]
 [4]
 [9]]


In [4]:
#image pre processing

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

In [ ]:
#CNN create

def create_CNN(input_observation):
      """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(input_observation, [-1, 80, 80, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  #pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=conv1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu)
  #pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(conv2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])}
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)
    return 

In [ ]:
#main loop

In [9]:

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel()
    delta_l2 = np.outer(delta_L, weights['2'])
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards

In [ ]:
## Architecture

# Take in inputs from the screen and preprocess them
# Pass them into an NN
# Update the weights of the NN using gradient descent
# weights['1'] - Matrix that holds weights of pixels passing into hidden layer. Dimensions: [200 x 80 x 80] -> [200 x 6400]
# weights['2'] - Matrix that holds weights of hidden layer passing into output. Dimensions: [1 x 200]

# Process is:

# processed_observations = image vector - [6400 x 1] array
# Compute hidden_layer_values = weights['1'] dot processed_observations ([200 x 6400] dot [6400 x 1]) -> [200 x 1] - this gives initial activation values.
# Next we need to transform those either via a sigmoid or an ReLU of some sort. Let's use ReLU
# ReLU(hidden_layer_values)
# Next we need to pass this one layer further
# output_layer_value = weights['2'] dot hidden_layer_values ([1 x 200] dot [200 x 1] -> [1 x 1])
# Now our output layer is the probability of going up or down. Let's make sure this output is between 0 and 1 by passing it through a sigmoid
# p = sigmoid(output_layer_value)

# Learning after round has finished:

# Figure out the result
# Compute the error
# Use the error to calculate the gradient
    # The below dimensions all assume we had exactly 10 frames in the round (not necessarily true!)
    # dC_dw2 = hidden_layer_values^T dot gradient_log_p ([1 x 2000] dot [2000 x 1] -> 1x1)
    # delta_1 = gradient_log_p outer_product weights['2'] = [2000 x 1] outer_product [1 x 200] ([2000 x 200])
    # dC_dw1 = delta_1^T dot input_observations ([200 x 2000]x dot [2000 x 64000] -> [200 x 64000])

# After some batch size of rounds has finished,
    # Use rmsprop to move weights['1'] and weights['2'] in the direction of the gradient
# Repeat!

import gym
import numpy as np

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel()
    delta_l2 = np.outer(delta_L, weights['2'])
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards


def main():
    env = gym.make("Pong-v0")
    observation = env.reset() # This gets us the image

    # hyperparameters
    episode_number = 0
    batch_size = 10
    gamma = 0.99 # discount factor for reward
    decay_rate = 0.99
    num_hidden_layer_neurons = 200
    input_dimensions = 80 * 80
    learning_rate = 1e-4

    episode_number = 0
    reward_sum = 0
    running_reward = None
    prev_processed_observations = None

    weights = {
        '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions),
        '2': np.random.randn(num_hidden_layer_neurons) / np.sqrt(num_hidden_layer_neurons)
    }

    # To be used with rmsprop algorithm (http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
    expectation_g_squared = {}
    g_dict = {}
    for layer_name in weights.keys():
        expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
        g_dict[layer_name] = np.zeros_like(weights[layer_name])

    episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []


    while True:
        env.render()
        processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
        hidden_layer_values, up_probability = apply_neural_nets(processed_observations, weights)
    
        episode_observations.append(processed_observations)
        episode_hidden_layer_values.append(hidden_layer_values)

        action = choose_action(up_probability)

        # carry out the chosen action
        observation, reward, done, info = env.step(action)

        reward_sum += reward
        episode_rewards.append(reward)

        # see here: http://cs231n.github.io/neural-networks-2/#losses
        fake_label = 1 if action == 2 else 0
        loss_function_gradient = fake_label - up_probability
        episode_gradient_log_ps.append(loss_function_gradient)


        if done: # an episode finished
            episode_number += 1

            # Combine the following values for the episode
            episode_hidden_layer_values = np.vstack(episode_hidden_layer_values)
            episode_observations = np.vstack(episode_observations)
            episode_gradient_log_ps = np.vstack(episode_gradient_log_ps)
            episode_rewards = np.vstack(episode_rewards)

            # Tweak the gradient of the log_ps based on the discounted rewards
            episode_gradient_log_ps_discounted = discount_with_rewards(episode_gradient_log_ps, episode_rewards, gamma)

            gradient = compute_gradient(
              episode_gradient_log_ps_discounted,
              episode_hidden_layer_values,
              episode_observations,
              weights
            )

            # Sum the gradient for use when we hit the batch size
            for layer_name in gradient:
                g_dict[layer_name] += gradient[layer_name]

            if episode_number % batch_size == 0:
                update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate)

            episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], [] # reset values
            observation = env.reset() # reset env
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
            reward_sum = 0
            prev_processed_observations = None

main()